In [10]:
%matplotlib inline
project_dir = '../../'
data_dir = project_dir + 'data/'

import sys
sys.path.append(project_dir + 'NN/')

import numpy as np
import importlib
import time
import matplotlib.pyplot as plt
import MLP
import layer
importlib.reload(MLP)
importlib.reload(layer)
from MLP import MLP
from utils.preprocessing import split,Normalize
from utils.losses import MSE, MEE

In [11]:
raw_data=np.loadtxt("../../data/TR.csv",delimiter=",")[:,1:]
#raw_data=np.delete(raw_data,10,axis=1)
np.random.shuffle(raw_data)
#raw_normalizer=Normalize()
#raw_data=raw_normalizer.normalize(raw_data)

frac_test=0.15
data, test_set=split(raw_data,kind="hold_out",frac_training=1-frac_test)

frac_training=0.8
training,validation=split(data,kind="hold_out",frac_training=frac_training)

input_data=training[:,1:-2]
labels=training[:,-2:]
val_data=validation[:,1:-2]
val_labels=validation[:,-2:]
print(len(training))

1004


In [12]:
import utils.grid_search
importlib.reload(utils.grid_search)
from utils.grid_search import grid_search

# Structure to test: 
n_feat = np.shape(labels)[1]
model1 = {'structure':[40, n_feat], 'func':['sigmoid', 'linear'], 'starting_points': [0.5]*2}
model2 = {'structure':[10, n_feat], 'func':['sigmoid','linear'], 'starting_points': [0.5]*2}
dict_models = {'Pippo' : model1, 
               'Pluto' : model2}

list_eta = [1e-1]
list_alpha = [1e-1, 1e-3, 0]
list_lamb = [1e-3, 0]
dict_params = {'eta':list_eta, 'alpha':list_alpha, 'lamb':list_lamb, 'epoch':[500] }

grid_results = grid_search(MLP, dict_models, dict_params, 
                           input_data, labels, val_data, val_labels, MEE,
                           verbose = 1)

0 on 12: Model Pippo --> Train parameters: {'eta': 0.1, 'alpha': 0.1, 'lamb': 0.001, 'epoch': 500}
Epoch 500:  [train MEE = 1.4087]  [val MEE = 1.4186]                              
Elapsed time: 4.5849833488464355 s


1 on 12: Model Pippo --> Train parameters: {'eta': 0.1, 'alpha': 0.1, 'lamb': 0, 'epoch': 500}
Epoch 500:  [train MEE = 1.3455]  [val MEE = 1.3575]                              
Elapsed time: 4.454357147216797 s


2 on 12: Model Pippo --> Train parameters: {'eta': 0.1, 'alpha': 0.001, 'lamb': 0.001, 'epoch': 500}
Epoch 500:  [train MEE = 1.4369]  [val MEE = 1.4425]                              
Elapsed time: 4.768038511276245 s


3 on 12: Model Pippo --> Train parameters: {'eta': 0.1, 'alpha': 0.001, 'lamb': 0, 'epoch': 500}
Epoch 500:  [train MEE = 1.3592]  [val MEE = 1.3744]                              
Elapsed time: 5.339097023010254 s


4 on 12: Model Pippo --> Train parameters: {'eta': 0.1, 'alpha': 0, 'lamb': 0.001, 'epoch': 500}
Epoch 500:  [train MEE = 1.4330]  

In [13]:
grid_results

[{'train': {'eta': 0.1, 'alpha': 0.001, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [10, 2],
   'func': ['sigmoid', 'linear'],
   'starting_points': [0.5, 0.5]},
  'model_name': 'Pluto',
  'Error': 1.3408770969787043},
 {'train': {'eta': 0.1, 'alpha': 0.1, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [10, 2],
   'func': ['sigmoid', 'linear'],
   'starting_points': [0.5, 0.5]},
  'model_name': 'Pluto',
  'Error': 1.3511701797394926},
 {'train': {'eta': 0.1, 'alpha': 0.1, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [40, 2],
   'func': ['sigmoid', 'linear'],
   'starting_points': [0.5, 0.5]},
  'model_name': 'Pippo',
  'Error': 1.3575391323684818},
 {'train': {'eta': 0.1, 'alpha': 0.001, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [40, 2],
   'func': ['sigmoid', 'linear'],
   'starting_points': [0.5, 0.5]},
  'model_name': 'Pippo',
  'Error': 1.374401308355167},
 {'train': {'eta': 0.1, 'alpha': 0, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [10, 2],
   'f